### References

In [1]:
# Authors: Eduardo S Vieira <edusvieirap@gmail.com>
# License: MIT

print(__doc__)

Automatically created module for IPython interactive environment


### Data loading

In [2]:
import pickle

import pandas as pd

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

from collections import Counter

from sklearn.preprocessing import StandardScaler

In [3]:
training_data = pd.read_csv("Datasets/training.csv")
testing_data = pd.read_csv("Datasets/testing.csv")

X_train = training_data.drop(["default payment next month"], axis=1)
y_train = training_data["default payment next month"]

X_test = testing_data.drop(["default payment next month"], axis=1)
y_test = testing_data["default payment next month"]

In [4]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_train.shape

(18000, 24)

In [5]:
scaler = StandardScaler()

X_test = scaler.fit_transform(X_test)

X_test.shape

(12000, 24)

In [6]:
print(f"The training data set is imbalanced: {Counter(y_train)}")
print(f"The testing data set is imbalanced: {Counter(y_test)}")

The training data set is imbalanced: Counter({0: 14092, 1: 3908})
The testing data set is imbalanced: Counter({0: 9272, 1: 2728})


### Metrics for evaluating model

Links for references metrics

    1. F1 Score: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

    2. Precision: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

    3. Recall: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

    4. Accuracy: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

    5. ROC AUC: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

    6. Coen´s Kappa: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html

In [7]:
class Metrics():
    def get_results(self, y_true, y_pred):
        f1 = f1_score(y_true, y_pred, zero_division=1, average="weighted")
        precision = precision_score(y_true, y_pred, zero_division=1, average="weighted")
        recall = recall_score(y_true, y_pred, zero_division=1, average="weighted")
        accuracy = accuracy_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred, average="weighted")
        kappa = cohen_kappa_score(y_true, y_pred)

        return f1, precision, recall, accuracy, auc, kappa

### Training

In [8]:
def generate_reports(best_result, method_name, step):
    pd.DataFrame(best_result).to_csv(f"Reports/No SMOTE/{method_name}-{step}-reports.csv", index=0)   

In [9]:
def train(method_name, model):
    kf = StratifiedKFold(n_splits=10)

    metrics = Metrics()
    n_fold = 1
    best_result = {"F1": 0, "Precision": 0, "Recall": 0, "Accuracy": 0, "AUC": 0}

    for train_index, test_index in kf.split(X_train, y_train):
        #X_kfold_train, X_kfold_test = X_train.iloc[train_index], X_train.iloc[test_index]
        X_kfold_train, X_kfold_test = X_train[train_index], X_train[test_index]
        y_kfold_train, y_kfold_test = y_train.iloc[train_index], y_train.iloc[test_index]

        m = model
                
        m.fit(X_kfold_train, y_kfold_train)

        y_pred = m.predict(X_kfold_test)

        f1, precision, recall, accuracy, auc, kappa = metrics.get_results(y_kfold_test, y_pred)

        print(f"Score at fold {n_fold}: F1-Score = {f1} Precision = {precision} Recall = {recall} Accuracy = {accuracy} AUC = {auc} Kappa = {kappa}")

        if best_result["F1"] < f1:
            best_result = {"F1": [f1], "Precision": [precision], "Recall": [recall], "Accuracy": [accuracy], "AUC": [auc], "Kappa": [kappa]}

            pickle.dump(m, open(f"Models/No SMOTE/{method_name}-model.sav", 'wb'))

            generate_reports(best_result, method_name, "train")

        n_fold += 1

In [220]:
methods = {
    "xgboost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    #"random-forest": RandomForestClassifier(),
    #"knn": KNeighborsClassifier(),
    #"decision-tree": DecisionTreeClassifier(),
    #"extra-trees": ExtraTreesClassifier(),
    #"ada-boost": AdaBoostClassifier(),
    #"gradient-boosting": GradientBoostingClassifier(),
    #"histogram-boosting": HistGradientBoostingClassifier()
}

for key in methods.keys():
    print("Evaluating method: " + key)
    train(method_name=key, model=methods[key])
    print("\n")

Evaluating method: random-forest
Score at fold 1: F1-Score = 0.7926282224233547 Precision = 0.7946774470030284 Recall = 0.815 Accuracy = 0.815 AUC = 0.6435624659028916 Kappa = 0.34504687602438855
Score at fold 2: F1-Score = 0.7819058474521897 Precision = 0.7785373489158092 Recall = 0.8016666666666666 Accuracy = 0.8016666666666666 AUC = 0.6350518276050192 Kappa = 0.3161356235234021
Score at fold 3: F1-Score = 0.7989494328885496 Precision = 0.8024953079764401 Recall = 0.8205555555555556 Accuracy = 0.8205555555555556 AUC = 0.6525541867316248 Kappa = 0.3663767736873079
Score at fold 4: F1-Score = 0.7965471254119959 Precision = 0.7980828246158684 Recall = 0.8172222222222222 Accuracy = 0.8172222222222222 AUC = 0.6513489278823202 Kappa = 0.36028449084176284
Score at fold 5: F1-Score = 0.7907852709360248 Precision = 0.7932340498823122 Recall = 0.8138888888888889 Accuracy = 0.8138888888888889 AUC = 0.6409045612875941 Kappa = 0.33990725861185667
Score at fold 6: F1-Score = 0.7963455173204936 Pre

### Testing

In [204]:
def test(method_name):
    metrics = Metrics()
    best_result = {"F1": 0, "Precision": 0, "Recall": 0, "Accuracy": 0, "AUC": 0}

    
    model = pickle.load(open(f"Models/No SMOTE/{method_name}-model.sav", 'rb'))
                
    y_pred = model.predict(X_test)

    f1, precision, recall, accuracy, auc, kappa = metrics.get_results(y_test, y_pred)

    print(f"Score at fold F1-Score = {f1} Precision = {precision} Recall = {recall} Accuracy = {accuracy} AUC = {auc} Kappa = {kappa}")

    best_result = {"Method": [method_name], "F1": [f1], "Precision": [precision], "Recall": [recall], "Accuracy": [accuracy], "AUC": [auc], "Kappa": [kappa]}
    
    generate_reports(best_result, method_name, "test")

In [205]:
for key in methods.keys():
    test(method_name=key)

Score at fold F1-Score = 0.7872396699601601 Precision = 0.7927147067054827 Recall = 0.8110833333333334 Accuracy = 0.8110833333333334 AUC = 0.6458101394416766 Kappa = 0.3516141100057545
Score at fold F1-Score = 0.7942840622415219 Precision = 0.8006728869134466 Recall = 0.8168333333333333 Accuracy = 0.8168333333333333 AUC = 0.6554815178420066 Kappa = 0.373595958183493
Score at fold F1-Score = 0.7039764691217739 Precision = 0.6889970108527131 Recall = 0.7423333333333333 Accuracy = 0.7423333333333333 AUC = 0.5376768955945943 Kappa = 0.0931664874376773
Score at fold F1-Score = 0.7306977110238746 Precision = 0.730645511511165 Recall = 0.73075 Accuracy = 0.73075 AUC = 0.6165927941217401 Kappa = 0.2332761171007982
Score at fold F1-Score = 0.789873427424248 Precision = 0.7949766600981663 Recall = 0.81275 Accuracy = 0.81275 AUC = 0.6501226218881178 Kappa = 0.3603274334032629
Score at fold F1-Score = 0.7884594158736102 Precision = 0.800701422879909 Recall = 0.81575 Accuracy = 0.81575 AUC = 0.6424

## Results

In [207]:
results = pd.DataFrame(columns=["F1", "Precision", "Recall", "Accuracy", "AUC", "Kappa"])

for key in methods.keys():
    results = pd.concat([results, pd.read_csv(f"Reports/No SMOTE/{key}-test-reports.csv")])
 
results.sort_values(by="AUC")

,F1,Precision,Recall,Accuracy,AUC,Kappa,Method
0,0.703976,0.688997,0.742333,0.742333,0.537677,0.093166,knn
0,0.730698,0.730646,0.730750,0.730750,0.616593,0.233276,decision-tree
0,0.788459,0.800701,0.815750,0.815750,0.642491,0.352068,ada-boost
0,0.787240,0.792715,0.811083,0.811083,0.645810,0.351614,xgboost
0,0.789873,0.794977,0.812750,0.812750,0.650123,0.360327,extra-trees
0,0.794159,0.803160,0.818250,0.818250,0.653035,0.371703,gradient-boosting
0,0.794284,0.800673,0.816833,0.816833,0.655482,0.373596,random-forest
0,0.796051,0.804773,0.819500,0.819500,0.656043,0.377881,histogram-boosting
